In [ ]:
#Bibliotecas
import pandas as pd
import numpy as np

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.impute import SimpleImputer # tratamento de valores ausentes
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer # definir tratamento para cada lista de variáveis
from sklearn.pipeline import Pipeline # definir sequência de cálculos do algoritmo de aprendizagem

#biliotecas do usuario
from transformadores import *
from utils import remover_outlier, buscar_outlier

#bibliotecas de visualização
from matplotlib import pyplot as plt
import seaborn as sns

# Omitir warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#importando os dados
dados = pd.read_csv('dados.csv',
                   sep = ',')

In [ ]:
dados.head()

In [ ]:
dados.columns

In [ ]:
print(f'Número de linhas: {dados.shape[0]}')
print(f'Número de colunas: {dados.shape[1]}')

<p>Primeiro, vamos definir qual é o problema que iremos solucionar.</p>
<p>Seguindo a ordem sugerida no escopo do desafio, temos</p>
<ol>
    <li>Estimar o faturamento</li>
    <li>Classificar o potencial</li>
    <li>Clusterizar por perfil e renda</li>
<ol>

<p>Analisando os dados disponíveis, temos que a variável potencial pode ser retirada. Esse pensamento se justifica, pois em um ambiente produtivo, teríamos um modelo estimando o faturamento e em seguida estimaríamos o potencial.</p>

In [ ]:
#dados utilizados para modelarmos o faturamento
dados_fat = dados.drop('potencial',
                       axis = 1)\
                .copy()

In [ ]:
dados_fat.head()

In [ ]:
dados_fat.columns

In [ ]:
print(f'Número de linhas: {dados_fat.shape[0]}')
print(f'Número de colunas: {dados_fat.shape[1]}')

<p>Como nosso problema consistirá em prever o faturamento por bairro em SP, trataremos os dados referentes ao mesmo como dados <i>out of sample</i>, ou seja, os dados de produção.</p>

In [ ]:
df = dados_fat[dados_fat.estado == 'RJ'].reset_index().drop('index',axis=1).copy()

In [ ]:
df.head()

In [ ]:
print(f'Número de linhas: {df.shape[0]}')
print(f'Número de colunas: {df.shape[1]}')

In [ ]:
df.info()

In [ ]:
df.rendaMedia = df.rendaMedia.astype('float')

<p>Temos então 160 observações para analisarmos, antes de inicarmos o processo de modelagem.</p>
<p>Vamos separar as variáveis que iremos utilizar. Nesse primeiro momento, podemos retirar as variáveis de código, nome, cidade e estado.</p>

In [ ]:
# Variáveis Numéricas
vnum = [x for x in df.columns if x not in ['codigo', 'nome', 'cidade','estado','faturamento']]
# Variável target
vtgt = ['faturamento']

<p>Fazendo uma análise nos tipos de dados de cada variável, temos que rendaMedia está como <i>object</i>. Antes que seguir a análise, vamos tranformá-la em <i>float</i>.</p>

<p>Podemos começar fazendo uma análise descritiva dos dados, buscando entender o comportamento e distribuição dos mesmos.</p>

In [ ]:
df[vnum].isnull().sum().sort_values(ascending=False)

In [ ]:
df[vnum].describe().T

<p> Observamos que temos 6 dados nulos na variável rendaMedia. Além disso, vemos uma distribuição muito esparsa dos dados, pois o valor mínimo é 645 e o máximo 63887.</p>
</p>Primeiro, vamos tratar os dados nulos. Existem várias forma de se trata um dado nulo, tais como:</p>
<ol>
    <li>Substituir por alguma estatística descritiva (Ex: Média, Mediana, Moda)</li>
    <li>Deletar as observações</li>
    <li>Estimar os valores utilizando um modelo.</li>
<ol>
    
<p>Por simplicidade e por temos poucos dados nulos, vamos utilizar a técnica de substituir os dados pela mediana. Essa decisão vem através da análise do boxplot abaixo.</p>
    <p>Podemos observar que os dados estão muito concentrados em valores próximos ao mínimo da observação da rendaMedia e existem valores muito discrepantes, por isso usar a estrátegia da median conseguiríamos trazer esse valores da rendaMedia para uma escala mais real.</p>

<p>Antes de fazermos o preenchimento de dados nulos, vamos retirar os <i>outliers</i>, pois podemos causar distorção no resultado do modelo no futuro. </p>

In [ ]:
sns.boxplot(x=df.dropna().rendaMedia, data=df, orient='h', palette="Blues")

In [ ]:
fig, axs = plt.subplots(7, 2, figsize = (20, 25), facecolor='white')  
for axs, v in zip(axs.flatten(), vnum):
    ax = sns.boxplot(x=v, data=df, orient='h', palette="Blues", ax=axs)

<p>Vamos investigar quais observações estão gerando essa distorção nos dados. Esses valores devem ser ajustados, pois o uso de de outliers causa distorções no modelo que não queremos.</p>

In [ ]:
dictt = {}
for col in df[vnum].columns:
    dictt[col] = buscar_outlier(df[col])


In [ ]:
[[key,len(value)] for key,value in dictt.items()]

In [ ]:
print(f'Quantidade de Outliers: {[(key,len(value)) for key,value in dictt.items()]}')

In [ ]:
# Método de imputação para variáveis numéricas -> substituir pela mediana
imp_num = SimpleImputer(missing_values=np.nan, strategy='median', add_indicator=False)

In [ ]:
transformador_numerico = Pipeline(steps = [('imputer', imp_num),
                                           ('nomes', NomeadorAtributos(colunas = vnum))], verbose = True)

In [ ]:
df_num  = transformador_numerico.fit_transform(df[vnum])

<p>Como estamos trabalahndo com valores muito discrepantes, vamos seguir a técnica de normalização das <i>features</i>. Usaremos para isso a transformação Min-Max.</p>

In [ ]:
#instanciando MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
#adicionando na pipeline 
transformador_numerico.steps.append(('normalizador', scaler))

In [ ]:
transformador_numerico.fit(df[vnum])

In [ ]:
df_num = pd.DataFrame(transformador_numerico.fit_transform(df[vnum]), columns = vnum)

In [ ]:
df_num.describe().T.sort_index()

In [ ]:
plt.figure(figsize = (20,20))
sns.heatmap(df[vtgt+vnum].corr(), annot = True, cmap = 'magma')

In [ ]:
df_new = pd.concat([df_num, df[vtgt]], axis = 1)

In [ ]:
df_new.head()

<p>Aplicando o VIF para retirar colinearidade.</p>

In [ ]:
df_vif = df_new.copy()
df_vif.drop('faturamento', axis = 1, inplace = True)
df_vif['constante'] = 1

In [ ]:
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(df_vif.values,i) for i in range(df_vif.shape[1])]
vif['features'] = df_vif.columns
vif.sort_values('VIF Factor', ascending = False)

<p>Para tratarmos nosso dados que podem vir nulos, vamos criar um imputer para preenche-los com a mediana.</p>

<p>Como os dados têm uma discrepância em escala, vamos fazer uma normalização dos dados para deixá-los em um range de (0,1).</p>

<p>Ao analisarmos o <i>heatmap</i> podemos ver uma grande correlação entra as variáveis de população.
Além disso, vemos que as variáveis de domicílios A1, A2, B1 junto com a rendaMedia são as variáveis que impactam positivamente o faturamento.</p>
<p>Para contornamos o problema da multicolienaridade de uma forma mais simple vamos utilizar algoritmos de floresta, como RandomForest.</p>

<p>O segundo passo é verificar se encontramos alguma colinearidade entre as variáveis.</p>

<p>Na célula abaixo será consolidado os diversos tratamentos aplicados a cada conjunto de variáveis.
Para as variáveis numéricas a necessidade de imputação pela média nos valores ausentes e para as variáveis categóricas a criação de variáveis indicadoras.</p>